In [1]:
import os
import numpy as np
import glob

import keras
from keras import models
from keras import layers
from keras import optimizers
from keras.layers import Dropout, Input
from keras.layers import Conv2D, Conv3D, Flatten, Dense, Reshape, BatchNormalization
from keras.models import Model
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint
from keras.layers import Conv3D, MaxPool3D, Flatten, Dense

import tensorflow as tf

Using TensorFlow backend.


In [2]:
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="3,4"

In [3]:
train_x = np.load('/home/ckwwk/data/germination_data/old_germination_data.npy')

In [4]:
train_x.shape

(1536, 60, 60, 275, 1)

In [5]:
t1 = train_x[286:1536,:,:,20:160]
tt1 = train_x[0:286,:,:,20:160]

In [6]:
import pandas as pd

label_df = pd.read_csv("/home/ckwwk/data/germination_data/label/old_germination_label.csv")
label_df.head(5)

,label
0,0
1,0
2,0
3,1
4,0


In [7]:
from keras.utils import to_categorical
train_y = label_df[286:1536]
train_y = to_categorical(train_y)

In [8]:
test_y = label_df[0:286]
test_y = to_categorical(test_y)

In [9]:
t1 = t1.astype('float32')
tt1 = tt1.astype('float32')

In [10]:
from keras.backend import tensorflow_backend as K
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
K.set_session(tf.Session(config=config))

In [11]:
input_layer = Input((60, 60, 140, 1))

## convolutional layers
conv_layer1 = Conv3D(filters=8, kernel_size=(3, 3, 3), activation='relu',padding='same')(input_layer)
conv_layer2 = Conv3D(filters=16, kernel_size=(3, 3, 3), activation='relu',padding='same')(conv_layer1)

## add max pooling to obtain the most imformatic features
pooling_layer1 = MaxPool3D(pool_size=(2, 2, 3))(conv_layer2)

conv_layer3 = Conv3D(filters=16, kernel_size=(3, 3, 3), activation='relu',padding='same')(pooling_layer1)
conv_layer4 = Conv3D(filters=32, kernel_size=(3, 3, 3), activation='relu',padding='same')(conv_layer3)
pooling_layer2 = MaxPool3D(pool_size=(2, 2, 3))(conv_layer4)

conv_layer5 = Conv3D(filters=32, kernel_size=(3, 3, 3), activation='relu',padding='same')(pooling_layer2)
conv_layer6 = Conv3D(filters=64, kernel_size=(3, 3, 3), activation='relu',padding='same')(conv_layer5)
pooling_layer3 = MaxPool3D(pool_size=(2, 2, 3))(conv_layer6)

## perform batch normalization on the convolution outputs before feeding it to MLP architecture
pooling_layer4 = BatchNormalization()(pooling_layer3)
flatten_layer = Flatten()(pooling_layer4)

## create an MLP architecture with dense layers : 4096 -> 512 -> 10
## add dropouts to avoid overfitting / perform regularization
dense_layer1 = Dense(units=1024, activation='relu')(flatten_layer)
dense_layer1 = Dropout(0.4)(dense_layer1)
dense_layer2 = Dense(units=256, activation='relu')(dense_layer1)
dense_layer2 = Dropout(0.4)(dense_layer2)
output_layer = Dense(units=2, activation='softmax')(dense_layer2)

## define the model with input layer and output layer
model = Model(inputs=input_layer, outputs=output_layer)

W1023 18:56:46.889865 140388783314752 deprecation_wrapper.py:119] From /home/ckwwk/.conda/envs/ck/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:66: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W1023 18:56:46.895560 140388783314752 deprecation_wrapper.py:119] From /home/ckwwk/.conda/envs/ck/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:541: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W1023 18:56:46.900637 140388783314752 deprecation_wrapper.py:119] From /home/ckwwk/.conda/envs/ck/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:4432: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W1023 18:56:47.150613 140388783314752 deprecation_wrapper.py:119] From /home/ckwwk/.conda/envs/ck/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:148: The name tf.placeholder_with_default is deprecated. Please use tf.com

In [12]:
model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 60, 60, 140, 1)    0         
_________________________________________________________________
conv3d_1 (Conv3D)            (None, 60, 60, 140, 8)    224       
_________________________________________________________________
conv3d_2 (Conv3D)            (None, 60, 60, 140, 16)   3472      
_________________________________________________________________
max_pooling3d_1 (MaxPooling3 (None, 30, 30, 46, 16)    0         
_________________________________________________________________
conv3d_3 (Conv3D)            (None, 30, 30, 46, 16)    6928      
_________________________________________________________________
conv3d_4 (Conv3D)            (None, 30, 30, 46, 32)    13856     
_________________________________________________________________
max_pooling3d_2 (MaxPooling3 (None, 15, 15, 15, 32)    0   

In [13]:
from keras.utils import multi_gpu_model
model = multi_gpu_model(model,gpus=2)

W1023 18:56:47.267843 140388783314752 deprecation_wrapper.py:119] From /home/ckwwk/.conda/envs/ck/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:190: The name tf.get_default_session is deprecated. Please use tf.compat.v1.get_default_session instead.



In [14]:
adam = Adam(lr=0.0001, decay=1e-06)
model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])

W1023 18:56:48.530960 140388783314752 deprecation_wrapper.py:119] From /home/ckwwk/.conda/envs/ck/lib/python3.7/site-packages/keras/optimizers.py:793: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.



In [15]:
filepath = "select_basic.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
tb = keras.callbacks.TensorBoard(log_dir='select_basic',histogram_freq=1,)
callbacks_list = [checkpoint,tb]

In [16]:
import timeit
start = timeit.default_timer()

history = model.fit(x=t1, y=train_y , batch_size=32, epochs=200,validation_split=0.1 ,callbacks=callbacks_list)

stop = timeit.default_timer()
print(stop - start)

W1023 18:56:49.596101 140388783314752 deprecation.py:323] From /home/ckwwk/.conda/envs/ck/lib/python3.7/site-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 1125 samples, validate on 125 samples


W1023 18:56:52.392243 140388783314752 deprecation_wrapper.py:119] From /home/ckwwk/.conda/envs/ck/lib/python3.7/site-packages/keras/callbacks.py:1120: The name tf.summary.histogram is deprecated. Please use tf.compat.v1.summary.histogram instead.

W1023 18:56:52.449263 140388783314752 deprecation_wrapper.py:119] From /home/ckwwk/.conda/envs/ck/lib/python3.7/site-packages/keras/callbacks.py:1122: The name tf.summary.merge_all is deprecated. Please use tf.compat.v1.summary.merge_all instead.

W1023 18:56:52.454104 140388783314752 deprecation_wrapper.py:119] From /home/ckwwk/.conda/envs/ck/lib/python3.7/site-packages/keras/callbacks.py:1125: The name tf.summary.FileWriter is deprecated. Please use tf.compat.v1.summary.FileWriter instead.



Epoch 1/200
1125/1125 [==============================] - 20s 18ms/step - loss: 0.7470 - acc: 0.5342 - val_loss: 0.6505 - val_acc: 0.6480

Epoch 00001: val_acc improved from -inf to 0.64800, saving model to select_basic.hdf5
Epoch 2/200
1125/1125 [==============================] - 13s 12ms/step - loss: 0.6777 - acc: 0.5902 - val_loss: 0.6705 - val_acc: 0.6480

Epoch 00002: val_acc did not improve from 0.64800
Epoch 3/200
1125/1125 [==============================] - 14s 12ms/step - loss: 0.6789 - acc: 0.6009 - val_loss: 0.6895 - val_acc: 0.5680

Epoch 00003: val_acc did not improve from 0.64800
Epoch 4/200
1125/1125 [==============================] - 14s 12ms/step - loss: 0.6797 - acc: 0.5902 - val_loss: 0.6744 - val_acc: 0.6480

Epoch 00004: val_acc did not improve from 0.64800
Epoch 5/200
1125/1125 [==============================] - 13s 12ms/step - loss: 0.6775 - acc: 0.6000 - val_loss: 0.6530 - val_acc: 0.6480

Epoch 00005: val_acc did not improve from 0.64800
Epoch 6/200
1125/1125 [=

In [17]:
# load best weights
model.load_weights("select_basic.hdf5")
model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])

In [19]:
#model.predict(test_x,batch_size=10)
model.evaluate(tt1,test_y,batch_size=32)

286/286 [==============================] - 2s 7ms/step


[1.307689650075419, 0.7412587408419256]